# COVID-19 Ländervergleich

## Ingo Dahn (dahn@dahn-research.eu)

Mit diesem Notebook kann man die Entwicklung der COVID-19-Daten (Infizierte und Verstorbene) in zwei Ländern vergleichen. Diese Seite wird unter der [Creative Commons Lizenz CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/) veröffentlicht.

Die Daten werden von https://opendata.ecdc.europa.eu/covid19/casedistribution/csv geladen und sind so stets aktuell.

In [3]:
import pandas as pd
url="https://opendata.ecdc.europa.eu/covid19/casedistribution/csv"
data=pd.read_csv(url)
data=data.sort_values(by=['countriesAndTerritories','year','month','day'])
dataV=data.values
dataT=data.T
dataTV=dataT.values
regions=sorted(set(dataTV[10]))
dataD={}
for n in range(0,len(dataV)):
   line=dataV[n]
   region=line[10];country=line[6];date=line[0]; cases=line[4]; deaths=line[5]; population=line[9]
   if country == 'Cura\xc3\xa7ao':
    country='Curacao'
   if region == 'Other' or population == 'nan':
       next
   if not region in dataD.keys():
               dataD[region]={}
   if not country in dataD[region].keys():
        dataD[region][country]={'population': population, 'dates':[], 'cases':[],'deaths':[]}
   dataD[region][country]['dates'].append(date)
   dataD[region][country]['cases'].append(cases)
   dataD[region][country]['deaths'].append(deaths)
defaultCountries={'Europe': 'Germany', 'America': 'United_States_of_America','Asia': 'China', 'Africa':'South_Africa', 'Oceania': 'Australia'}
datesAll=dataD['Europe']['Germany']['dates']
dataNr=len(datesAll)
datesMap={}
for n in range(0,dataNr):
    datesMap[datesAll[n]]=n
def setDates(Dates,Data):
    LL=[0 for n in range(0,dataNr)]
    for i in range(0,len(Dates)):
        LL[datesMap[Dates[i]]]=Data[i]
    return LL
regions=sorted(defaultCountries.keys())
def cumulateList(L):
    LL=[];s=0;
    for i in range(0,len(L)):
        s += L[i]
        LL.append(s)
    return LL
def normListPer(L,Ord,Pop):
    po=Ord/Pop
    return [n*po for n in L]
def cumulatedNormed(L,c,n,P):
    if c:
        casePlotData1=cumulateList(L)
    else:
        casePlotData1=L
    if n:
        casePlotData=normListPer(casePlotData1,10^5,P)
    else:
        casePlotData=casePlotData1
    return casePlotData
region_1='Europe';country_1='Germany'
region_2='America'; country_2='United_States_of_America'
show(html(u"Daten von %s bis %s geladen"%(datesAll[0],datesAll[dataNr-1])))
@interact
def panel_11(region_1s=selector(regions,default=region_1,buttons=True,label="Region 1:")):
    @interact
    def panel_12(country_1s=selector(sorted(dataD[region_1s].keys()),default=defaultCountries[region_1s], label="Land 1:")):
        global region_1
        region_1=region_1s
        global country_1
        country_1=country_1s

        @interact
        def panel_21(region_2s=selector(regions,default=region_2,buttons=True,label="Region 2:")):
            @interact
            def panel_22(country_2s=selector(sorted(dataD[region_2s].keys()),default=defaultCountries[region_2s], label="Land 2:")):
                global region_2
                region_2=region_2s
                global country_2
                country_2=country_2s
                @interact
                def _(cumulate1=checkbox(True,label='Werte kumulieren'), norm1=checkbox(True,label="Werte normieren")):
                    global cumulate
                    cumulate=cumulate1
                    global norm
                    norm=norm1
                    C1=setDates(dataD[region_1][country_1]['dates'],dataD[region_1][country_1]['cases'])
                    C2=setDates(dataD[region_2][country_2]['dates'],dataD[region_2][country_2]['cases'])
                    D1=setDates(dataD[region_1][country_1]['dates'],dataD[region_1][country_1]['deaths'])
                    D2=setDates(dataD[region_2][country_2]['dates'],dataD[region_2][country_2]['deaths'])
                    DataCasePlot1=cumulatedNormed(C1,cumulate1,norm1,dataD[region_1][country_1]['population'])
                    DataCasePlot2=cumulatedNormed(C2,cumulate1,norm1,dataD[region_2][country_2]['population'])
                    DataDeathPlot1=cumulatedNormed(D1,cumulate1,norm1,dataD[region_1][country_1]['population'])
                    DataDeathPlot2=cumulatedNormed(D2,cumulate1,norm1,dataD[region_2][country_2]['population'])
                    labelCase1="COVID-19-Infektionen in "+country_1
                    labelCase2="COVID-19-Infektionen in "+country_2
                    labelDeath1="COVID-19-Tote in "+country_1
                    labelDeath2="COVID-19-Tote in "+country_2
                    if cumulate:
                        labelCase1 += " kumuliert";labelCase2 += " kumuliert";labelDeath1 += " kumuliert";labelDeath2 += " kumuliert";
                    if norm:
                        labelCase1 += ", pro 100 000 Einwohner";labelCase2 += ", pro 100 000 Einwohner";labelDeath1 += ", pro 100 000 Einwohner";labelDeath2 += ", pro 100 000 Einwohner";
                    show(list_plot(DataCasePlot1, color='blue',legend_label=labelCase1)+list_plot(DataCasePlot2, color='red',legend_label=labelCase2))
                    show(list_plot(DataDeathPlot1, color='blue',legend_label=labelDeath1)+list_plot(DataDeathPlot2, color='red',legend_label=labelDeath2))

Daten von 31/12/2019 bis 07/06/2020 geladen

SW50ZXJhY3RpdmUgZnVuY3Rpb24gPGZ1bmN0aW9uIHBhbmVsXzExIGF0IDB4N2YxZTA0Y2I1Mzk4PiB3aXRoIDEgd2lkZ2V0CiAgcmVnaW9uXzFzOiBUb2dnbGVCdXR0b25zKGRlc2NyaXB0aW/igKY=
